In [6]:
import json
from my_utils import NERDataset,collate_fn
from torch.utils.data import Dataset,DataLoader
import jieba

In [3]:
dataset = NERDataset('nlp2024-data/dataset/dev.json')
dataloader = DataLoader(dataset,batch_size=8,collate_fn=collate_fn)
for batch in dataloader:
    print(batch)
    break

([[2584, 825, 2075, 1147, 1814, 1019, 122, 1515, 1147, 2448, 737, 2670, 926, 2448, 737, 2670, 1983, 970, 899, 564, 2075, 899, 1518, 737], [1147, 1814, 2670, 570, 2501, 2485, 737, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961], [2670, 2738, 1123, 2696, 2448, 737, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961], [2670, 2959, 2501, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961], [1983, 2738, 870, 2075, 864, 1983, 926, 899, 1518, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961], [2747, 1542, 2670, 2093, 210, 2448, 737, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961], [2670, 1983, 1549, 2604, 2696, 1455, 737, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 2961, 296

In [4]:
## 获取所有字符
data_dev = json.load(open('nlp2024-data/dataset/dev.json',encoding='utf-8'))
data_train = json.load(open('nlp2024-data/dataset/train.json',encoding='utf-8'))
data_test = json.load(open('nlp2024-data/dataset/test.json',encoding='utf-8'))
def get_text_set(data):
    text_set = set()
    for key,val in data.items():
        for character in val['diagnosis']:
            text_set.add(character)
        for character in val['self_report']:
            text_set.add(character)
        dialogues = val['dialogue']
        for dia in dialogues:
            for character in dia['speaker']:
                text_set.add(character)
            for character in dia['sentence']:
                text_set.add(character)
            for word in dia['symptom_norm']:
                for character in word:
                    text_set.add(character)
    return text_set
text_set_train = get_text_set(data_train)
text_set_dev = get_text_set(data_dev)
text_set_test = get_text_set(data_test)
text_set = text_set_dev | text_set_train | text_set_test
len(text_set)
text_list = list(text_set)
text_list.append('#')
text_dict = {text_list[i]:i for i in range(len(text_list))}
with open('char2id.json','w',encoding='utf-8') as f:
    json.dump(text_dict,f,ensure_ascii=False)

In [9]:
# 获取所有的词
def get_split_word(data):
    word_set = set()
    for key,val in data.items():
        word_set = word_set | set(jieba.lcut(val['diagnosis']))
        word_set = word_set | set(jieba.lcut(val['self_report']))
        dialogues = val['dialogue']
        for dia in dialogues:
            word_set = word_set | set(jieba.lcut(dia['sentence']))
            word_set = word_set | set(dia['symptom_norm'])
    return word_set
data_dev = json.load(open('nlp2024-data/dataset/dev.json',encoding='utf-8'))
data_train = json.load(open('nlp2024-data/dataset/train.json',encoding='utf-8'))
data_test = json.load(open('nlp2024-data/dataset/test.json',encoding='utf-8'))
word_set_train = get_split_word(data_train)
word_set_dev = get_split_word(data_dev)
word_set_test = get_split_word(data_test)
word_set = word_set_dev | word_set_train | word_set_test
word_list = list(word_set)
word_list.append('#')
word_dict = {word_list[i]:i for i in range(len(word_list))}
with open('word2id.json','w',encoding='utf-8') as f:
    json.dump(word_dict,f,ensure_ascii=False)

In [33]:
# 获取所有的词
word_dict = dict()
def add_to_dict(l):
    for w in l:
        if w in word_dict:
            word_dict[w] += 1
        else:
            word_dict[w] = 0
def get_split_word(data):
    word_set = set()
    for key,val in data.items():
        l =  jieba.lcut(val['diagnosis'])
        add_to_dict(l)
        l = jieba.lcut(val['self_report'])
        add_to_dict(l)
        dialogues = val['dialogue']
        for dia in dialogues:
            add_to_dict(jieba.lcut(dia['sentence']) + dia['symptom_norm'])
data_dev = json.load(open('nlp2024-data/dataset/dev.json',encoding='utf-8'))
data_train = json.load(open('nlp2024-data/dataset/train.json',encoding='utf-8'))
data_test = json.load(open('nlp2024-data/dataset/test.json',encoding='utf-8'))
word_set_train = get_split_word(data_train)
word_set_dev = get_split_word(data_dev)
word_set_test = get_split_word(data_test)
len(word_dict)
word_items = sorted(list(word_dict.items()),key=lambda x:x[1],reverse=True)
word_set = {word_items[i][0] for i in range(len(word_items)) if word_items[i][1] > 15 }

In [34]:
word_list = list(word_set)
word_list.append('#')
word_dict = {word_list[i]:i for i in range(len(word_list))}
with open('word2id.json','w',encoding='utf-8') as f:
    json.dump(word_dict,f,ensure_ascii=False)

In [2]:
from transformers import BertModel,BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')

/Users/lijinliang/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/lijinliang/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
text = '你好'
inputs = tokenizer(text,return_tensors='pt',padding=True,truncation=True)
inputs

{'input_ids': tensor([[ 101,  872, 1962,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [13]:
out = model(**inputs)

In [16]:
out.last_hidden_state.shape

torch.Size([1, 4, 768])

In [3]:
from my_utils import NERDatasetBert, collate_fn_bert
datasetBert = NERDatasetBert('nlp2024-data/dataset/dev.json',tokenizer)
dataloader = DataLoader(datasetBert,batch_size=2, collate_fn=collate_fn_bert)


In [5]:
for batch in dataloader:
    inputs = {"input_ids":batch[0],
              "attention_mask":batch[1]}
    out = model(**inputs)
    print(out)
    break

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.7955, -0.0604, -0.5145,  ..., -0.0065, -0.0573,  0.0602],
         [ 0.0069, -0.1325,  0.1473,  ..., -0.6525, -0.4242,  0.1753],
         [ 0.8572,  0.2717,  0.3732,  ..., -0.1347,  0.5311, -0.4706],
         ...,
         [ 0.6514, -0.5964, -0.2139,  ...,  0.6149,  0.1953,  0.6304],
         [ 0.5118,  0.4172, -0.5241,  ..., -0.5596,  0.4079, -0.0406],
         [ 0.0385, -0.4536, -0.0262,  ..., -0.3393, -0.2469,  0.2462]],

        [[ 0.4655, -0.4097, -0.0354,  ...,  0.1277,  0.0083, -0.1648],
         [ 0.0356, -0.2946, -0.5383,  ..., -0.3654, -0.5864,  0.3523],
         [ 0.5439, -1.2687, -0.8399,  ...,  0.5940, -0.0624,  0.2715],
         ...,
         [ 0.2470, -0.0931, -0.1315,  ..., -0.3311, -0.1926, -0.1254],
         [ 0.1547, -0.4363, -0.2219,  ..., -0.1255, -0.2254, -0.0903],
         [ 0.1825, -0.3251, -0.1521,  ..., -0.1516, -0.2679, -0.1133]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [2]:
import pandas as pd
data = pd.read_csv('nlp2024-data/dataset/symptom_norm.csv')
data.to_dict()

{'norm': {0: '咳嗽',
  1: '发热',
  2: '感冒',
  3: '鼻流涕',
  4: '腹泻',
  5: '痰',
  6: '呕吐',
  7: '中等度热',
  8: '稀便',
  9: '高热',
  10: '消化不良',
  11: '肺炎',
  12: '支气管炎',
  13: '鼻塞',
  14: '病毒感染',
  15: '低热',
  16: '黄疸',
  17: '细菌感染',
  18: '哭闹',
  19: '喷嚏',
  20: '便秘',
  21: '退热',
  22: '呼吸道感染',
  23: '腹胀',
  24: '喘息',
  25: '脱水',
  26: '干咳',
  27: '过敏',
  28: '咽喉炎',
  29: '绿便',
  30: '精神软',
  31: '皮疹',
  32: '水样便',
  33: '出汗',
  34: '屁',
  35: '上呼吸道感染',
  36: '支气管肺炎',
  37: '肠炎',
  38: '支原体感染',
  39: '腹痛',
  40: '嗓子沙哑',
  41: '大便粘液',
  42: '痰鸣音',
  43: '食欲不振',
  44: '炎症',
  45: '恶心',
  46: '气喘',
  47: '气管炎',
  48: '咳痰',
  49: '干呕',
  50: '扁桃体炎',
  51: '幼儿急疹',
  52: '嗜睡',
  53: '湿疹',
  54: '尿量减少',
  55: '上火',
  56: '脓血便',
  57: '抽搐',
  58: '胃肠功能紊乱',
  59: '四肢厥冷',
  60: '肠鸣音',
  61: '疱疹',
  62: '头痛',
  63: '鼻炎',
  64: '缺钙',
  65: '贫血',
  66: '血便',
  67: '过敏体质',
  68: '咽喉不适',
  69: '过敏性咳嗽',
  70: '大便酸臭',
  71: '呼吸困难',
  72: '电解质紊乱',
  73: '病理性黄疸',
  74: '口腔溃疡',
  75: '蛋花汤样便',
  76: '烦躁',
  77: '充血

In [1]:
import spacy

In [10]:
nlp = spacy.load('zh_core_web_lg')
text = '我有一个苹果'
doc = nlp(text)
token_range = dict()
for token in doc:
    token_range[token.i] = [token.idx,token.idx + len(token.text)]
token_range

{0: [0, 1], 1: [1, 2], 2: [2, 4], 3: [4, 6]}

In [12]:
import jieba
text = "欢迎来到华农"
jieba.lcut(text)

['欢迎', '来到', '华农']

In [7]:
for token in doc:
    print(token.i,token.idx,token.text,token.dep_,token.head,token.head.idx)

0 0 我 dep 有 1
1 1 有 ROOT 有 1
2 2 一个 dep 有 1
3 4 苹果 ROOT 苹果 4


0

In [27]:
char_to_token_id = {}
doc = nlp(text)
for token in doc:
    token_id = token.i
    start_char = token.idx
    end_char = start_char + len(token.text)
    for char_id in range(start_char, end_char):
        char_to_token_id[char_id] = token_id

In [35]:
import torch

In [36]:
a = torch.tensor([[1,2,3],[4,5,6]])
a


tensor([[1, 2, 3],
        [4, 5, 6]])

In [38]:
m = torch.tensor([[1,0,0],[0,0,1]])

In [40]:
a[m==True]

tensor([1, 6])

In [8]:
def insert_asterisks(text, labels):
    entity_pos = []
    tokens = list(text)
    new_tokens = []
    i = 0
    while(i < len(tokens)):
        if labels[i]=="B-Symptom":
            e = [len(new_tokens)]
            new_tokens.append('*')
            begin = i
            end = i+1
            while end < len(tokens) and labels[end]=="I-Symptom":
                end += 1
            e.append(e[0]+end-begin+1)
            entity_pos.append(e)
            for j in range(begin,end):
                new_tokens.append(tokens[j])
            new_tokens.append('*')
            i = end-1
        else:
            new_tokens.append(tokens[i])
        i += 1
    return new_tokens,entity_pos

text = "磨牙，晚上翻来覆去，大便干，吃的多，很容易积食"
labels = "B-Symptom I-Symptom O O O O O O O O O O O O O O O O O O O B-Symptom I-Symptom".split()

modified_text, entity_positions = insert_asterisks(text, labels)

print("Modified text:", modified_text)
print("Entity positions:", entity_positions)


Modified text: ['*', '磨', '牙', '*', '，', '晚', '上', '翻', '来', '覆', '去', '，', '大', '便', '干', '，', '吃', '的', '多', '，', '很', '容', '易', '*', '积', '食', '*']
Entity positions: [[0, 3], [23, 26]]


In [10]:
modified_text[26]

'*'

In [30]:
from transformers import BartTokenizer, T5ForConditionalGeneration, Text2TextGenerationPipeline
from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline

# 加载模型和tokenizer
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
model = BartForConditionalGeneration.from_pretrained("fnlp/bart-base-chinese")
text2text_generator = Text2TextGenerationPipeline(model, tokenizer)

# 准备输入文本
input_data = r"""
    "input":"我叫张三，今年45岁，最近咳嗽了很久，没有既往史，主诉：[MASK]，现病史：[MASK]"
"""
# 将输入数据格式化为文本
input_text = r"""
{
    "主诉": "[MASK]",
    "现病史": "[MASK]",
    "辅助检查": "[MASK]",
    "既往史": "[MASK]",
    "诊断": "[MASK]",
    "建议": "[MASK]"
            
}
"""

# 生成医疗报告
result = text2text_generator(input_data, max_length=2000,min_length=200, do_sample=False)

# 打印生成的报告
print(result[0]['generated_text'])


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BartTokenizer'.


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [17]:
with open('test.txt','w') as f:
    f.write(result[0]['generated_text'])

In [26]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
input_data = r"""
{
    "input":{
{'title': '7个月的宝宝喉咙有痰十多天了，后面几天开始咳嗽，还有一天晚上烧到38度，去医院输了三天液之后咳嗽减轻了，但是痰还是很严重', 'context': '医生:您好，宝宝现在体温正常了吗？
医生:宝宝嗓子哑吗？有没有气喘？什么时候咳嗽频繁？
患者:现在正常了
医生:好的
患者:看了两个医生一个说是毛细支气管炎，一个说是支气管炎到支气管肺炎之间
患者:没有住院
医生:嗯嗯，现在还输液吗？
患者:明天还要输一次
患者:但我觉得没效果，要怎么办
医生:宝宝咳嗽比之前是减轻了吧？
患者:恩
患者:减轻了
患者:还有桔贝合剂
医生:这个是中成药止咳化痰
患者:氨茶碱片
医生:这个是平喘药物
患者:那还需要用什么药呢
医生:输液是什么?_?
医生:克林霉素就是控制感染的哦，喜炎平是止咳化痰的
患者:谢谢了
医生:积极配合医生治疗，跟医生多沟通孩子病情。
医生:注意清淡饮食，给宝贝多喝水，避免呛咳
}
}
"""
# 加载预训练的GPT-2模型和tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-chinese")
model = GPT2LMHeadModel.from_pretrained("gpt2-chinese")
inputs = tokenizer.encode("生成医疗报告"+input_data, return_tensors="pt")

# 生成文本
outputs = model.generate(
    inputs,
    max_length=2000,   # 设置生成文本的最大长度
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    length_penalty=2.0,
    early_stopping=True
)

# 解码生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 打印生成的报告
print(generated_text)

OSError: gpt2-chinese is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`